## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,41.11,70,20,34.52,few clouds
1,1,Praia,CV,14.9215,-23.5087,79.97,82,85,3.53,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,40.66,81,75,16.11,moderate rain
3,3,Jamestown,US,42.0970,-79.2353,72.72,73,100,5.75,overcast clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,70.56,94,83,7.00,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Praia,CV,14.9215,-23.5087,79.97,82,85,3.53,overcast clouds
3,3,Jamestown,US,42.0970,-79.2353,72.72,73,100,5.75,overcast clouds
4,4,Puerto Ayora,EC,-0.7393,-90.3518,70.56,94,83,7.00,broken clouds
6,6,Vaini,TO,-21.2000,-175.2000,78.96,83,20,9.22,few clouds
8,8,Albany,US,42.6001,-73.9662,76.59,65,100,1.61,overcast clouds
15,15,Kalaleh,IR,37.3807,55.4916,76.60,88,100,9.22,overcast clouds
17,17,Puerto Escondido,MX,15.8500,-97.0667,79.63,73,75,5.75,broken clouds
21,21,Tabou,CI,4.4230,-7.3528,75.13,77,91,10.94,overcast clouds
23,23,Bethel,US,41.3712,-73.4140,79.03,66,0,6.91,clear sky
32,32,Axim,GH,4.8699,-2.2405,75.97,84,100,11.23,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                190
City                   190
Country                190
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Praia,CV,79.97,overcast clouds,14.9215,-23.5087,
3,Jamestown,US,72.72,overcast clouds,42.0970,-79.2353,
4,Puerto Ayora,EC,70.56,broken clouds,-0.7393,-90.3518,
6,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
8,Albany,US,76.59,overcast clouds,42.6001,-73.9662,
15,Kalaleh,IR,76.60,overcast clouds,37.3807,55.4916,
17,Puerto Escondido,MX,79.63,broken clouds,15.8500,-97.0667,
21,Tabou,CI,75.13,overcast clouds,4.4230,-7.3528,
23,Bethel,US,79.03,clear sky,41.3712,-73.4140,
32,Axim,GH,75.97,overcast clouds,4.8699,-2.2405,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
#https://www.w3schools.com/python/pandas/ref_df_iterrows.asp
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]

    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# hotel_url = base_url + "&q=" + "boston"
# hotels = requests.get(base_url, params=params)
# hotels
# hotels = requests.get(base_url, params=params)
# hotels.json()

In [10]:
# 7. Drop the rows where there is no Hotel Name.
#https://stackoverflow.com/questions/29314033/drop-rows-containing-
#empty-cells-from-a-pandas-dataframe
clean_hotel_df = hotel_df.dropna()
clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Praia,CV,79.97,overcast clouds,14.9215,-23.5087,LT Aparthotel
3,Jamestown,US,72.72,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
4,Puerto Ayora,EC,70.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
6,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Kalaleh,IR,76.60,overcast clouds,37.3807,55.4916,کلاله
...,...,...,...,...,...,...,...
682,Muncar,ID,77.70,overcast clouds,-8.4333,114.3333,Zaky Homestay
685,Sept-Iles,CA,73.40,scattered clouds,50.2001,-66.3821,Comfort Inn
688,Wang Saphung,TH,78.82,few clouds,17.3010,101.7685,โรงแรมวังรีสอร์ท
689,Noshiro,JP,77.83,clear sky,40.2039,140.0242,Suigetsu


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
!jupyter nbextension enable --py widgetsnbextension
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
              max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))